In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime  
import time
from hashlib import sha1
import hmac
from wsgiref.handlers import format_date_time
from datetime import datetime
from time import mktime
import base64

In [2]:
app_id = '2d16769d481b4ab69ec6ecb0ccd0672b'
app_key = 'dbr3zEbzwX8cj1Yv7ZxGzb4IpAc'
class Auth():

    def __init__(self, app_id, app_key):
        self.app_id = app_id
        self.app_key = app_key

    def get_auth_header(self):
        xdate = format_date_time(mktime(datetime.now().timetuple()))
        hashed = hmac.new(self.app_key.encode('utf8'), ('x-date: ' + xdate).encode('utf8'), sha1)
        signature = base64.b64encode(hashed.digest()).decode()

        authorization = 'hmac username="' + self.app_id + '", ' + \
                        'algorithm="hmac-sha1", ' + \
                        'headers="x-date", ' + \
                        'signature="' + signature + '"'
        return {
            'Authorization': authorization,
            'x-date': format_date_time(mktime(datetime.now().timetuple())),
            'Accept - Encoding': 'gzip'
        }

In [ ]:
city=['NewTaipei','Taoyuan','Taichung','Tainan','Kaohsiung','Keelung','Hsinchu','HsinchuCounty','MiaoliCounty','ChanghuaCounty'
,'NantouCounty','YunlinCounty','ChiayiCounty','Chiayi','PingtungCounty','YilanCounty','HualienCounty','TaitungCounty'
,'KinmenCounty','PenghuCounty','LienchiangCounty','Taipei']
Bus_information=pd.DataFrame(columns = ["站牌唯一識別代碼","站牌代碼(為原資料內碼)","站牌中文名稱","站牌英文名稱"
                                        ,"站牌緯度(WGS84)","站牌經度(WGS84)","地理空間編碼","站牌地址","資料更新日期時間"
                                        ,"資料版本編號",'經此站牌唯一識別代碼','經此站牌代碼(為原資料內碼)','經此站牌中文名稱'
                                        ,"經此站牌英文名稱","經此路線識別代碼","經此路線代碼","經此路線中文","經此路線英文"])
for j in city:
    #print(j)
    a = Auth(app_id, app_key)
    r=requests.get("https://ptx.transportdata.tw/MOTC/v2/Bus/Station/City/"+str(j)+"?$top=5&$format=xml", headers= a.get_auth_header())
    r.close()
    soup = BeautifulSoup(r.text, "html.parser")
    for i in soup.find_all('busstation'): #每次跑一個站的資料
        #print(soup.prettify())
        print(j)
        #for j in i.find_all("stops"):
        stationuid = [stationuid.text for stationuid in i.find_all('stationuid')]

        stationid = [stationid.text for stationid in i.find_all('stationid')]

        StopName = [i.zh_tw.text for i in i.find_all('stationname')]

        #En = [en.text for en in soup.find_all('en')]

        PositionLat = [positionlat.text for positionlat in i.find_all('positionlat')]

        PositionLon = [positionlon.text for positionlon in i.find_all('positionlon')]

        GeoHash = [geohash.text for geohash in i.find_all('geohash')]

        Stopaddress = [stopaddress.text for stopaddress in i.find_all('stationaddress')]
        
        UpdateTime = [updatetime.text for updatetime in i.find_all('updatetime')]
        VersionID = [versionid.text for versionid in i.find_all('versionid')]
        
        Bus={'站牌唯一識別代碼':stationuid,'站牌代碼(為原資料內碼)':stationid,'站牌中文名稱':StopName
            ,"站牌緯度(WGS84)":PositionLat,"站牌經度(WGS84)":PositionLon
            ,"地理空間編碼":GeoHash,"站牌地址":Stopaddress,"資料更新日期時間":UpdateTime,"資料版本編號":VersionID}
        #print(Bus_information)
        Bus = pd.DataFrame.from_dict(Bus, orient='index')
        Bus=Bus.transpose()#把欄位顛倒過來
        
        #合併經過公車
        stationuid = [stationuid.text for stationuid in i.find_all('stationuid')]
        stopuid = [i.stopuid.text for i in i.find_all('stop')]
        stopid = [i.stopid.text for i in i.find_all('stop')]
        stopname_zh_tw=[i.stopname.zh_tw.text for i in i.find_all('stop')]
        stopname_en = [i.stopname.en.text for i in i.find_all('stop')]
        routeuid = [i.routeuid.text for i in i.find_all('stop')]
        routeid=[i.routeid.text for i in i.find_all('stop')]
        routename_zh_tw = [i.routename.zh_tw.text for i in i.find_all('stop')]
        routename_en=[i.routename.en.text for i in i.find_all('stop')]
        Bus_connection={'站牌唯一識別代碼':stationuid,'經此站牌唯一識別代碼':stopuid,'經此站牌代碼(為原資料內碼)':stopid,'經此站牌中文名稱':stopname_zh_tw
                ,"經此站牌英文名稱":stopname_en ,"經此路線識別代碼":routeuid,"經此路線代碼":routeid
                ,"經此路線中文":routename_zh_tw,"經此路線英文":routename_en}
        df = pd.DataFrame.from_dict(Bus_connection, orient='index')
        df=df.transpose()#把欄位顛倒過來
        df=df.fillna(method='pad')#用前一個值彌補後一個值
        _all=pd.merge(Bus, df,on="站牌唯一識別代碼")#完整資料
        print(_all)
        Bus_information=pd.concat([Bus_information, _all])
        #break
    
    break